## Introduction

We're going to spend a bit of time talking about a powerful method in timeseries analysis: **ARIMA**.  In order to begin talking about this, we need to cover a few concepts.  More information on this and much more can be found in the excellent resource [Forecasting: Principles and Practice](https://otexts.com/fpp3/); we've distilled a number of things from this resource and others into a more bite-sized post.

In order to talk about **ARIMA**, we're going to need a few concepts:

1. **Autoregression** (the _AR_ part of _ARIMA_),
2. **n-Diff** operations,
    - The process of "un-doing" differencing is called **Integration** (which is the _I_ in _ARIMA_),
3. **Moving Averages**, windowed averages of a timeseries (the _MA_ part of _ARIMA_).

---

Let's import all our stuff and proceed.  Don't worry if you don't know what all of this does, we'll be covering it in the post.

In [63]:
import altair as alt
import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA


# -- HELPER FUNCTIONS --
def plot_in_altair(df: pd.DataFrame) -> alt.Chart:
    """Plots df vals in altair."""
    # Altair likes long data.
    df_timeseries_melted = df.melt(
        value_vars=df.columns, 
        ignore_index=False
    )

    # Create the chart with an optional selection on the legend.
    selection = alt.selection_multi(fields=["variable"], bind='legend')
    chart = (
        alt.Chart(df_timeseries_melted.reset_index()).mark_line()
        .encode(x="index:T", y="value", color="variable",
                opacity=alt.condition(selection, alt.value(1), alt.value(0.2)))
        .add_selection(selection)
    )
    return chart

def make_data() -> pd.DataFrame:
    """Creates synthetic data for us to use."""
    time_domain_1 = pd.date_range('1980-01-01', freq="1m", periods=96)
    values = {
        "data_1": np.sin(np.linspace(0, 1920 * np.pi, 96)),
        "data_2": np.random.normal(size=96) + 
            np.concatenate([np.linspace(0, 5, 40), np.linspace(5, -5, 20), 
            np.linspace(0, 5, 36)]),
        "data_3": np.random.normal(size=96)
    }
    df_timeseries = pd.DataFrame(values, index=time_domain_1)
    return df_timeseries



## Timeseries, Trend, Seasonality

A **timeseries** can be thought of as quantative variables being plotted on a timeline.  For example, the following three datasets are good examples:

In [64]:
# See the data creation function at the top near the imports!
df = make_data()
plot_in_altair(df).interactive()

alt.Chart(...)

You can click on each of the variables in the legend above to show off that plot off (hold shift + click to select multiples).  Let's look at a few of these and see what the deal is.


``data_1`` is quite regular: it repeats in a predictable pattern.  This is **seasonality**: the presence of variations that occur at specific regular intervals.  

``data_2`` shows something which is similar, but not quite the same: it's rising in a predictable way, but the data is not predictable in the same way as ``data_1``.  This is a behavior which has similar variations as seasonality, but does _not_ occur at regular intervals &mdash; we call behavior like this a **trend**.  The a stock's hourly or daily prices are a good example of a timeseries with a trend: there are rises and falls, but we cannot predict (in the long term) when either will happen.

``data_3`` shows neither pattern: we see neither trends nor seasonality.  We will call such a timeseries **stationary**: it shows neither seasonality nor trends.  Think of white-noise here.

Take a look at [the plot in this chapter](https://otexts.com/fpp3/stationarity.html) and try to guess which are seasonal, which have trends, and which are stationary.  The answers are below the figures.  It's tricky!

## How to Make a Timeseries Stationary

There are some cool things we can do if we know that a timeseries is stationary.  We'll see this in a little bit but, for a second, let's talk about one of the most popular ways to make a (general) timeseries stationary.

Suppose we have some point ``y_t`` in our timeseries, and we look at `y_t+1`.  What could happen?  The value could have gone up, gone down, or stayed the same.  This feels like a pretty benign observation, but it turns out that it's quite useful to have a feature like this.  We call this ``1-diff``-ing, taking the difference between successive points to see a rate-of-change in our timeseries.  Let's look at our plots from above with 1-diff applied.



In [66]:
df = make_data()
df["data_1__1_diff"] = df["data_1"].diff(1)
df["data_2__1_diff"] = df["data_2"].diff(1)
df["data_3__1_diff"] = df["data_3"].diff(1)

plot_in_altair(df).interactive()

alt.Chart(...)

This plot looks fairly messy, so remember that you can click on the legend to show and hide selective timeseries values.

Looking at ``data_1`` and ``data_1__1_diff`` we don't see a huge change.  It's still extremely seasonal, definitely not stationary.  Bummer.  Let's go onward.

``data_2__1_diff`` is much more stationary: it's lost the trend completely (_for those of you who are math-enthusiasts: why did this happen?_) and now looks quite stationary (like white-noise).

``data_3__1_diff`` is also stationary, but because ``data_3`` was also stationary perhaps this isn't so surprising.

On real timeseries data, this method of making a "stationary feature" for your timeseries is quite effective.  It is also effective to take 2-diffs if a 1-diff has not made the data stationary enough.  Note that some series may be resiliant: they may not become stationary easily, as we saw above in our ``data_1`` example.  Nevertheless, this works for many real-world timeseries.

(Recall from the introduction that the reverse of this process is called _Integration_, which is the _I_ part of _ARIMA_.)

## Autoregression: Looking to the past to predict the future.

An **autoregressive model** for a timeseries is a linear combination of _past values_ and constant coefficients to calculate the current value.  In math terms, for a **p-autoregressive model**,

$$y_t = c_{1}y_{t-1} + c_{2}y_{t-2} + \cdots + c_{p}y_{t-p}$$

We will refer to _p_-autoregressions as ``AR(p)``.

Let's do an example of this, using ``data_3`` from above.

In [68]:
df = make_data()

# Calculate some autoregressions.
df.drop(["data_1", "data_2"], axis=1, inplace=True)
df["AR(01)"] = AutoReg(df["data_3"], 1).fit().predict()
df["AR(05)"] = AutoReg(df["data_3"], 5).fit().predict()
df["AR(10)"] = AutoReg(df["data_3"], 10).fit().predict()
df["AR(25)"] = AutoReg(df["data_3"], 25).fit().predict()

plot_in_altair(df).interactive()

alt.Chart(...)

Notice that as we get larger values of _p_, we get a better estimate of our timeseries.  This makes sense, since we're using more past data to predict a current value.  The downside is: we lose a lot of data at the beginning, since we don't have enough points to "look back on" and calculate our regression.  Compare ``AR(01)`` and ``AR(25)`` in the above plot.

What do we gain by doing an autoregression?  Recall that an autoregression takes in some past values and predicts a current value: we can use this type of model to perform regression-type analysis on our timeseries, just as we would do if we had a regression on non-timeseries data.  Powerful!

## Moving Averages

Moving averages in timeseries take certain windows of observations and take an average of them.  For example, a moving average with a window size of 5 taking a mean will look at 5 observations, take the mean of those observations, then shift up one timestep and do the same thing.  Plotting those means gives us the moving average of our timeseries.

Pandas makes this straight-forward:

In [70]:
df = make_data()
df.drop(["data_1", "data_3"], axis=1, inplace=True)

df["data_2__ma_05"] = df["data_2"].rolling(5, center=True).mean()
df["data_2__ma_10"] = df["data_2"].rolling(10, center=True).mean()
df["data_2__ma_20"] = df["data_2"].rolling(20, center=True).mean()

plot_in_altair(df).interactive()

alt.Chart(...)

Notice above that we started with something "spikey" (``data_2``) and as we used a bigger window for our moving average, our resulting data was "smoother".  The degree to which this moving average represents the original data is dependent on the window size, the amount of detail one needs to predict, and other similar restrictions.  Nevertheless, the moving average (sometimes called a _rolling average_) is a useful feature on its own with a number of parameters one can tune, including window size, window type, and type of average used.

## Putting Everything Together.

We've seen Autoregressive models (AR), we've seen diffing when we took our 1- and 2-diffs which is the "opposite" of Integration (I), and we've seen Moving Averages (MA) &mdash; looks like we're ready to check out ARIMA!

It should come as no surprise at this point that in order to create our ARIMA model, we're going to need to combine all of the thing we've learned above.

We follow the literature and say ``ARIMA(p,d,q)`` to mean an ARIMA model with

- ``p``-autoregressive (**AR(p)** in our notation above),
- ``d`` is the degree of diff-ing (``d``-diff in our notation),
- ``q`` is the order of the moving average part (the window size),

With these, check out the [Table 9.1 in this chapter](https://otexts.com/fpp3/non-seasonal-arima.html), notice the special cases.  Make sure you can reason out why these particular values create the associated result.  

For example, taking ``ARIMA(0, 1, 0)`` should result in a random walk.  Why?  Let's look.  We are using no autoregression or moving averages, but we are taking a 1-diff.  As we saw above, this is a way to make our model stationary (similar to white-noise) which is essentially a random walk.

As usual, Python makes it dead-easy to do ARIMA on our timeseries, as we see below.  (Ignore the warning for now...)

In [72]:
df = make_data()
df.drop(["data_1", "data_3"], axis=1, inplace=True)

# -- ARIMA --
arima_model = ARIMA(df, order=(5,1,5))
model_fit = arima_model.fit()
df["ARIMA(5,1,5)"] = model_fit.predict()

plot_in_altair(df).interactive()

/home/james/.cache/pypoetry/virtualenvs/blog-Mz1OaUtd-py3.9/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


alt.Chart(...)

This gives us a fairly close fit, even with somewhat arbitrary starting paramers!  In a future post, we'll discuss how to look for "good" starting parameters, as well as what to do once we have this model.